# Tests extracción de rasgos estilométricos con RAID

In [17]:
# import numpy as np
# import math
# import random
# import matplotlib.pyplot as plt
# import cvxopt
# import seaborn as sns
# import soundfile as sf
# import itertools
# import pywt
# import librosa
# import librosa.display
# from IPython.display import Audio
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score
# from cvxopt import matrix, solvers
# import pandas as pd
# import os
# import scipy
# import scipy.signal
# from sklearn.decomposition import PCA, TruncatedSVD, FactorAnalysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.model_selection import train_test_split
# from sklearn.random_projection import SparseRandomProjection, johnson_lindenstrauss_min_dim, GaussianRandomProjection
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# import torch
# from torch import nn
# import torch.optim as optim
# import torch.nn.functional as F
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

In [35]:
from raid import run_detection, run_evaluation
from raid.utils import load_data
import stylo_metrix as sm
import pandas as pd

from processer import split_text_into_sentences

## Manejo de RAID

In [20]:
# Download the RAID dataset without adversarial attacks
or_train_noadv_df = load_data(split="train", include_adversarial=False)
# test_noadv_df = load_data(split="test", include_adversarial=False)
# extra_noadv_df = load_data(split="extra", include_adversarial=False)

In [21]:
display(or_train_noadv_df.head(7))

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...
5,72fe360b-cce6-4daf-b66a-1d778f5964f8,72fe360b-cce6-4daf-b66a-1d778f5964f8,72fe360b-cce6-4daf-b66a-1d778f5964f8,human,NaN,NaN,none,abstracts,Segmenter: Transformer for Semantic Segmentation,NaN,Image segmentation is often ambiguous at the l...
6,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,human,NaN,NaN,none,abstracts,Mining Contextual Information Beyond Image for...,NaN,This paper studies the context aggregation pro...


In [22]:
intCols = ['model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
# train_noadv_df = or_train_noadv_df[or_train_noadv_df['model'] != 'human']

# Copia del dataframe con columnas específicas
train_noadv_df = or_train_noadv_df.copy()
train_noadv_df = train_noadv_df[intCols]

display(train_noadv_df.head(7))

,model,domain,title,prompt,generation
0,human,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,human,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,human,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,human,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,human,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...
5,human,abstracts,Segmenter: Transformer for Semantic Segmentation,NaN,Image segmentation is often ambiguous at the l...
6,human,abstracts,Mining Contextual Information Beyond Image for...,NaN,This paper studies the context aggregation pro...


In [23]:
print("Información del dataset de entrenamiento:")
print(f"Forma del dataset: {train_noadv_df.shape}")
print(f"Columnas: {list(train_noadv_df.columns)}")
print(f"Modelos unicos: {train_noadv_df['model'].unique()}")
print(f"Dominios unicos: {train_noadv_df['domain'].unique()}")

Información del dataset de entrenamiento:
Forma del dataset: (467985, 5)
Columnas: ['model', 'domain', 'title', 'prompt', 'generation']
Modelos unicos: ['human' 'llama-chat' 'mpt' 'mpt-chat' 'gpt2' 'mistral' 'mistral-chat'
 'gpt3' 'cohere' 'chatgpt' 'gpt4' 'cohere-chat']
Dominios unicos: ['abstracts' 'books' 'news' 'poetry' 'recipes' 'reddit' 'reviews' 'wiki']


In [24]:
# intCols = ['model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
filtered = train_noadv_df[train_noadv_df['model'] != 'human']

display(filtered.head(7))

,model,domain,title,prompt,generation
493,llama-chat,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,Write the abstract for the academic paper titl...,"In the paper ""FUTURE-AI: Guiding Principles an..."
494,llama-chat,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,Write the abstract for the academic paper titl...,"In the paper ""Future-AI: Guiding Principles an..."
495,llama-chat,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,Write the abstract for the academic paper titl...,"In this paper, we present EdgeFlow, a novel ap..."
496,llama-chat,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,Write the abstract for the academic paper titl...,"In this paper, we present a novel approach to ..."
497,llama-chat,abstracts,Semi-supervised Contrastive Learning for Label...,Write the abstract for the academic paper titl...,"In this paper, we propose a novel approach to ..."
498,llama-chat,abstracts,Semi-supervised Contrastive Learning for Label...,Write the abstract for the academic paper titl...,"In this paper, we propose a novel approach to ..."
499,llama-chat,abstracts,Combo Loss: Handling Input and Output Imbalanc...,Write the abstract for the academic paper titl...,"In the field of medical image segmentation, im..."


In [25]:
# Opción 1: Configurar pandas para mostrar más contenido
pd.set_option('display.max_colwidth', None)  # Sin límite de ancho
pd.set_option('display.max_rows', None)      # Sin límite de filas (usar con cuidado)

complex_filter = train_noadv_df[(train_noadv_df['model'] == 'human') &
                                (train_noadv_df['domain'] == 'abstracts')]

# Ahora al mostrar el dataframe, verás el contenido completo
display(complex_filter[['model', 'domain', 'generation']].head(3))

# Si quieres restaurar los valores por defecto de pandas:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

,model,domain,generation
0,human,abstracts,"The recent advancements in artificial intelligence (AI) combined with the\nextensive amount of data generated by today's clinical systems, has led to the\ndevelopment of imaging AI solutions across the whole value chain of medical\nimaging, including image reconstruction, medical image segmentation,\nimage-based diagnosis and treatment planning. Notwithstanding the successes and\nfuture potential of AI in medical imaging, many stakeholders are concerned of\nthe potential risks and ethical implications of imaging AI solutions, which are\nperceived as complex, opaque, and difficult to comprehend, utilise, and trust\nin critical clinical applications. Despite these concerns and risks, there are\ncurrently no concrete guidelines and best practices for guiding future AI\ndevelopments in medical imaging towards increased trust, safety and adoption.\nTo bridge this gap, this paper introduces a careful selection of guiding\nprinciples drawn from the accumulated experiences, consensus, and best\npractices from five large European projects on AI in Health Imaging. These\nguiding principles are named FUTURE-AI and its building blocks consist of (i)\nFairness, (ii) Universality, (iii) Traceability, (iv) Usability, (v) Robustness\nand (vi) Explainability. In a step-by-step approach, these guidelines are\nfurther translated into a framework of concrete recommendations for specifying,\ndeveloping, evaluating, and deploying technically, clinically and ethically\ntrustworthy AI solutions into clinical practice."
1,human,abstracts,"High-quality training data play a key role in image segmentation tasks.\nUsually, pixel-level annotations are expensive, laborious and time-consuming\nfor the large volume of training data. To reduce labelling cost and improve\nsegmentation quality, interactive segmentation methods have been proposed,\nwhich provide the result with just a few clicks. However, their performance\ndoes not meet the requirements of practical segmentation tasks in terms of\nspeed and accuracy. In this work, we propose EdgeFlow, a novel architecture\nthat fully utilizes interactive information of user clicks with edge-guided\nflow. Our method achieves state-of-the-art performance without any\npost-processing or iterative optimization scheme. Comprehensive experiments on\nbenchmarks also demonstrate the superiority of our method. In addition, with\nthe proposed method, we develop an efficient interactive segmentation tool for\npractical data annotation tasks. The source code and tool is avaliable at\nhttps://github.com/PaddlePaddle/PaddleSeg."
2,human,abstracts,"The success of deep learning methods in medical image segmentation tasks\nheavily depends on a large amount of labeled data to supervise the training. On\nthe other hand, the annotation of biomedical images requires domain knowledge\nand can be laborious. Recently, contrastive learning has demonstrated great\npotential in learning latent representation of images even without any label.\nExisting works have explored its application to biomedical image segmentation\nwhere only a small portion of data is labeled, through a pre-training phase\nbased on self-supervised contrastive learning without using any labels followed\nby a supervised fine-tuning phase on the labeled portion of data only. In this\npaper, we establish that by including the limited label in formation in the\npre-training phase, it is possible to boost the performance of contrastive\nlearning. We propose a supervised local contrastive loss that leverages limited\npixel-wise annotation to force pixels with the same label to gather around in\nthe embedding space. Such loss needs pixel-wise computation which can be\nexpensive for large images, and we further propose two strategies, downsampling\nand block division, to address the issue. We evaluate our methods on two public\nbiomedical image datasets of different modalities. With different amounts of\nlabeled data, our methods c

## Uso de Stylometrix

### Test de extracción

In [48]:
# Get sample dataframe 'generation'
generation_sample = filtered[['model', 'domain', 'generation']].sample(n=5, random_state=36)
display(generation_sample)

,model,domain,generation
407409,mpt,wiki,"She became a Christian at 12 years old, under ..."
79972,mistral,books,"Two teenage misfits meet at Camp Crystal Lake,..."
240222,gpt4,poetry,"In the quiet corners of the heart, where love'..."
55767,mpt-chat,abstracts,The paper proposes a novel approach for visual...
292999,chatgpt,recipes,Ingredients:\n- 1 small watermelon\n- 1 small ...


#### Extracción de rasgos en un registro

In [49]:
print(generation_sample['generation'].iloc[0])
outputList = split_text_into_sentences(generation_sample['generation'].iloc[0])

She became a Christian at 12 years old, under John Sargent's ministry; attended schools for girls in Philadelphia and Newburyport Massachusetts (graduating near or before age 13); was one year engaged to Henry Ward Beecher who introduced her by correspondence with Harriet Martineau.[2] After this disappointment she studied music until 1851 when Lyman Abbott called on James Miller McKim as he took trips through upstate towns preaching that it might bring them together again[citation needed], which did indeed occur within ten days but then continued their search after marriage,[1][3]. During Sarah Abbot McClellan lived outside Boston among various people helping slaves escape via Underground railroad into Canada while being pursued herself since 1850-1854 due because abolitionist views held deep inside heart made dangerous task taking risks during those troubled times especially considering fact slave status only legally differentiates person whom lives born America white color considere

In [50]:
for sentence in outputList:
    print(sentence)
    print("-----")

She became a Christian at 12 years old, under John Sargent's ministry; attended schools for girls in Philadelphia and Newburyport Massachusetts (graduating near or before age 13); was one year engaged to Henry Ward Beecher who introduced her by correspondence with Harriet Martineau.[2] After this disappointment she studied music until 1851 when Lyman Abbott called on James Miller McKim as he took trips through upstate towns preaching that it might bring them together again[citation needed], which did indeed occur within ten days but then continued their search after marriage,[1][3].
-----
During Sarah Abbot McClellan lived outside Boston among various people helping slaves escape via Underground railroad into Canada while being pursued herself since 1850-1854 due because abolitionist views held deep inside heart made dangerous task taking risks during those troubled times especially considering fact slave status only legally differentiates person whom lives born America white color con

In [53]:
stylo = sm.StyloMetrix('en', debug=True, save_path="testVisual/single") # define langauge, one of ('de','en', 'pl', 'ru', 'ukr')
metrics = stylo.transform(outputList) # list of texts
metrics

100%|██████████| 2/2 [00:02<00:00,  1.37s/it]

File saved in location: testVisual\single\sm_output1.csv
File saved in location: testVisual\single\sm_debug1.csv


(                                                text  POS_VERB  POS_NOUN  \
 0  She became a Christian at 12 years old, under ...  0.168539  0.146067   
 1  During Sarah Abbot McClellan lived outside Bos...  0.239264  0.343558   
 
     POS_ADJ   POS_ADV   POS_DET  POS_INTJ  POS_CONJ  POS_PART   POS_NUM  ...  \
 0  0.033708  0.033708  0.022472  0.000000  0.067416  0.011236  0.056180  ...   
 1  0.125767  0.092025  0.012270  0.009202  0.024540  0.009202  0.006135  ...   
 
     RE       ASF       ASM        OM  RCI       DMC        OR       QAS   PA  \
 0  0.0  0.000000  0.011236  0.000000  0.0  0.000000  0.000000  0.000000  0.0   
 1  0.0  0.003067  0.018405  0.006135  0.0  0.015337  0.003067  0.027607  0.0   
 
          PR  
 0  0.011236  
 1  0.000000  
 
 [2 rows x 197 columns],
                                                 text  \
 0  She became a Christian at 12 years old, under ...   
 1  During Sarah Abbot McClellan lived outside Bos...   
 
                                

#### Extracción de rasgos en múltiples registros

In [54]:
stylo = sm.StyloMetrix('en', debug=True, save_path="testVisual/multiple")
for idx, i in enumerate(generation_sample.index):
    print(f"Processing index: {i} (iteration {idx})")
    text = generation_sample['generation'].loc[i]
    sentences = split_text_into_sentences(text)
    
    # Cambiar los nombres de los archivos para cada iteración
    stylo.output_name = f"output_iter{idx}_idx{i}"
    stylo.debug_name = f"debug_iter{idx}_idx{i}"
    features = stylo.transform(sentences)
    print(f"Saved: {stylo.output_name}.csv and {stylo.debug_name}.csv")


c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Processing index: 407409 (iteration 0)
[OK] Total de oraciones: 2


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 2/2 [00:02<00:00,  1.27s/it]



File saved in location: testVisual\multiple\output_iter0_idx4074091.csv
File saved in location: testVisual\multiple\debug_iter0_idx4074091.csv
Saved: output_iter0_idx407409.csv and debug_iter0_idx407409.csv
Processing index: 79972 (iteration 1)
[OK] Total de oraciones: 7


100%|██████████| 7/7 [00:02<00:00,  3.13it/s]



...

File saved in location: testVisual\multiple\output_iter2_idx2402221.csv
File saved in location: testVisual\multiple\debug_iter2_idx2402221.csv
Saved: output_iter2_idx240222.csv and debug_iter2_idx240222.csv
Processing index: 55767 (iteration 3)
[OK] Total de oraciones: 3


100%|██████████| 3/3 [00:00<00:00,  6.34it/s]



File saved in location: testVisual\multiple\output_iter3_idx557671.csv
File saved in location: testVisual\multiple\debug_iter3_idx557671.csv
Saved: output_iter3_idx55767.csv and debug_iter3_idx55767.csv
Processing index: 292999 (iteration 4)
[OK] Total de oraciones: 18


100%|██████████| 18/18 [00:02<00:00,  7.83it/s]

File saved in location: testVisual\multiple\output_iter4_idx2929991.csv
File saved in location: testVisual\multiple\debug_iter4_idx2929991.csv
Saved: output_iter4_idx292999.csv and debug_iter4_idx292999.csv
